# Installation

The following instructions were prepared using

In [1]:
versioninfo()

Julia Version 1.5.2
Commit 539f3ce943 (2020-09-23 23:17 UTC)
Platform Info:
  OS: Linux (x86_64-pc-linux-gnu)
  CPU: Intel(R) Core(TM) i9-9900X CPU @ 3.50GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-9.0.1 (ORCJIT, skylake-avx512)
Environment:
  JULIA_NUM_THREADS = 4


Before exploring the notebook you need to clone the main repository:
```bash
 git clone https://github.com/kalmarek/1812.03456.git
```
This notebook should be located in `1812.03456/notebooks` directory.

In the main directory (`1812.03456`) you should run the following code in `julia`s `REPL` console to instantiate the environment for computations:
```julia
using Pkg
Pkg.activate(".")
Pkg.instantiate()
```
(this needs to be done once per installation). Jupyter notebook may be launched then from `REPL` by
```
julia> using IJulia

julia> notebook(dir=".")

```

Instantiation should install (among others): the [`SCS` solver][1], [`JuMP` package][2] for mathematical programming and `IntervalArithmetic.jl` package from [`ValidatedNumerics.jl`][3].

The environment uses [`Groups.jl`][7], [`GroupRings.jl`][6] (which are built on the framework of  [`AbstractAlgebra.jl`][4]) and [`PropertyT.jl`][8] packages.

[1]: https://github.com/cvxgrp/scs  
[2]: https://github.com/JuliaOpt/JuMP.jl  
[3]: https://github.com/JuliaIntervals/ValidatedNumerics.jl
[4]: https://github.com/Nemocas/AbstractAlgebra.jl
[5]: https://github.com/Nemocas/Nemo.jl
[6]: https://github.com/kalmarek/GroupRings.jl
[7]: https://github.com/kalmarek/Groups.jl
[8]: https://github.com/kalmarek/PropertyT.jl

# The computation

The following programme certifies that
$$\operatorname{Adj}_4 + \operatorname{Op}_4 - 0.82\Delta_4 =\Sigma_i \xi_i^*\xi_i \in \Sigma^2_2\mathbb{R}\operatorname{SL}(4,\mathbb{Z}).$$

With small changes (which we will indicate) it also certifies that 
$$\operatorname{Adj}_3 - 0.157999\Delta_3 \in \Sigma^2_2\mathbb{R}\operatorname{SL}(3,\mathbb{Z})$$
and that
$$\operatorname{Adj}_5 +1.5 \mathrm{Op}_5 - 1.5\Delta_5 \in \Sigma^2_2\mathbb{R}\operatorname{SL}(5,\mathbb{Z}).$$

In [2]:
using Pkg
Pkg.activate("..")
using Dates
now()

 Activating environment at `~/1812.03456/Project.toml`


2020-10-21T13:02:26.066

In [3]:
using LinearAlgebra
using AbstractAlgebra
using Groups
using GroupRings
using PropertyT

So far we only made the needed packages available in the notebook. 
In the next cell we define `G` to be the set of all $4\times 4$ matrices over $\mathbb Z$.
(For the second computation, set `N=3` below; for the third, set `N=5`)

In [4]:
N = 4
G = MatrixAlgebra(zz, N)

Matrix Algebra of degree 4 over Integers

## Generating set
Now we create the elementary matrices $E_{i,j}$. The set of all such matrices and their inverses is denoted by `S`.

In [5]:
S = PropertyT.generating_set(G)

24-element Array{AbstractAlgebra.Generic.MatAlgElem{Int64},1}:
 [1 1 0 0; 0 1 0 0; 0 0 1 0; 0 0 0 1]
 [1 0 1 0; 0 1 0 0; 0 0 1 0; 0 0 0 1]
 [1 0 0 1; 0 1 0 0; 0 0 1 0; 0 0 0 1]
 [1 0 0 0; 1 1 0 0; 0 0 1 0; 0 0 0 1]
 [1 0 0 0; 0 1 1 0; 0 0 1 0; 0 0 0 1]
 [1 0 0 0; 0 1 0 1; 0 0 1 0; 0 0 0 1]
 [1 0 0 0; 0 1 0 0; 1 0 1 0; 0 0 0 1]
 [1 0 0 0; 0 1 0 0; 0 1 1 0; 0 0 0 1]
 [1 0 0 0; 0 1 0 0; 0 0 1 1; 0 0 0 1]
 [1 0 0 0; 0 1 0 0; 0 0 1 0; 1 0 0 1]
 [1 0 0 0; 0 1 0 0; 0 0 1 0; 0 1 0 1]
 [1 0 0 0; 0 1 0 0; 0 0 1 0; 0 0 1 1]
 [1 -1 0 0; 0 1 0 0; 0 0 1 0; 0 0 0 1]
 [1 0 -1 0; 0 1 0 0; 0 0 1 0; 0 0 0 1]
 [1 0 0 -1; 0 1 0 0; 0 0 1 0; 0 0 0 1]
 [1 0 0 0; -1 1 0 0; 0 0 1 0; 0 0 0 1]
 [1 0 0 0; 0 1 -1 0; 0 0 1 0; 0 0 0 1]
 [1 0 0 0; 0 1 0 -1; 0 0 1 0; 0 0 0 1]
 [1 0 0 0; 0 1 0 0; -1 0 1 0; 0 0 0 1]
 [1 0 0 0; 0 1 0 0; 0 -1 1 0; 0 0 0 1]
 [1 0 0 0; 0 1 0 0; 0 0 1 -1; 0 0 0 1]
 [1 0 0 0; 0 1 0 0; 0 0 1 0; -1 0 0 1]
 [1 0 0 0; 0 1 0 0; 0 0 1 0; 0 -1 0 1]
 [1 0 0 0; 0 1 0 0; 0 0 1 0; 0 0 -1 1]

## Group Ring and Laplacians
Now we will generate the ball `E_R` of radius $R=4$ in $\operatorname{SL}(N,\mathbb{Z})$ and use this as a (partial) basis in a group ring (denoted by `RG` below). Such group ring also needs a multiplication table (`pm`, which is actually a *division table*) which is created as follows: when $x,y$ reside at positions `i`-th and `j`-th in `E_R`, then `pm[i,j] = k`, where `k` is the position of $x^{-1}y$ in `E_R`. 

In [6]:
halfradius = 2
E_R, sizes = Groups.wlmetric_ball(S, radius=2*halfradius);
E_rdict = GroupRings.reverse_dict(E_R)
pm = GroupRings.create_pm(E_R, E_rdict, sizes[halfradius]; twisted=true);
RG = GroupRing(G, E_R, E_rdict, pm)
@show sizes;
Δ = length(S)*one(RG) - sum(RG(s) for s in S)

sizes = [25, 433, 6149, 75197]


24[1 0 0 0; 0 1 0 0; 0 0 1 0; 0 0 0 1] - 1[1 1 0 0; 0 1 0 0; 0 0 1 0; 0 0 0 1] - 1[1 0 1 0; 0 1 0 0; 0 0 1 0; 0 0 0 1] - 1[1 0 0 1; 0 1 0 0; 0 0 1 0; 0 0 0 1] - 1[1 0 0 0; 1 1 0 0; 0 0 1 0; 0 0 0 1] - 1[1 0 0 0; 0 1 1 0; 0 0 1 0; 0 0 0 1] - 1[1 0 0 0; 0 1 0 1; 0 0 1 0; 0 0 0 1] - 1[1 0 0 0; 0 1 0 0; 1 0 1 0; 0 0 0 1] - 1[1 0 0 0; 0 1 0 0; 0 1 1 0; 0 0 0 1] - 1[1 0 0 0; 0 1 0 0; 0 0 1 1; 0 0 0 1] - 1[1 0 0 0; 0 1 0 0; 0 0 1 0; 1 0 0 1] - 1[1 0 0 0; 0 1 0 0; 0 0 1 0; 0 1 0 1] - 1[1 0 0 0; 0 1 0 0; 0 0 1 0; 0 0 1 1] - 1[1 -1 0 0; 0 1 0 0; 0 0 1 0; 0 0 0 1] - 1[1 0 -1 0; 0 1 0 0; 0 0 1 0; 0 0 0 1] - 1[1 0 0 -1; 0 1 0 0; 0 0 1 0; 0 0 0 1] - 1[1 0 0 0; -1 1 0 0; 0 0 1 0; 0 0 0 1] - 1[1 0 0 0; 0 1 -1 0; 0 0 1 0; 0 0 0 1] - 1[1 0 0 0; 0 1 0 -1; 0 0 1 0; 0 0 0 1] - 1[1 0 0 0; 0 1 0 0; -1 0 1 0; 0 0 0 1] - 1[1 0 0 0; 0 1 0 0; 0 -1 1 0; 0 0 0 1] - 1[1 0 0 0; 0 1 0 0; 0 0 1 -1; 0 0 0 1] - 1[1 0 0 0; 0 1 0 0; 0 0 1 0; -1 0 0 1] - 1[1 0 0 0; 0 1 0 0; 0 0 1 0; 0 -1 0 1] - 1[1 0 0 0; 0 1 0 0; 0 0 1 0;

## Orbit Decomposition
Now something happens: in the next cell we split the subspace of $\mathbb{R} \operatorname{SL}(N, \mathbb{Z})$ supported on `E_R` into irreducible representations of the wreath product $\mathbb Z / 2 \mathbb Z \wr \operatorname{Sym}_N$. The action of wreath product on the elements of the matrix space is by conjugation, i.e. permutation of rows and columns.
We also compute projections on the invariant subspaces to later speed up the optimisation step.

In [7]:
block_decomposition = let bd = PropertyT.BlockDecomposition(RG, WreathProduct(SymmetricGroup(2), SymmetricGroup(N)))
    PropertyT.decimate(bd, false);
end;

┌ Info: Decomposing basis of RG into orbits of
│   autS = Wreath Product of Full symmetric group over 2 elements by Full symmetric group over 4 elements
└ @ PropertyT /home/kalmar/.julia/packages/PropertyT/vcGsE/src/blockdecomposition.jl:15


  0.304335 seconds (1.10 M allocations: 89.684 MiB, 8.60% gc time)


┌ Info: The action has 558 orbits
└ @ PropertyT /home/kalmar/.julia/packages/PropertyT/vcGsE/src/blockdecomposition.jl:18
┌ Info: Finding projections in the Group Ring of
│   autS = Wreath Product of Full symmetric group over 2 elements by Full symmetric group over 4 elements
└ @ PropertyT /home/kalmar/.julia/packages/PropertyT/vcGsE/src/blockdecomposition.jl:20


  3.818060 seconds (6.02 M allocations: 308.271 MiB, 2.65% gc time)


┌ Info: Finding AutS-action matrix representation
└ @ PropertyT /home/kalmar/.julia/packages/PropertyT/vcGsE/src/blockdecomposition.jl:23


  0.136971 seconds (582.79 k allocations: 52.493 MiB)
  0.492913 seconds (1.51 M allocations: 93.189 MiB, 8.70% gc time)


┌ Info: Computing the projection matrices Uπs
└ @ PropertyT /home/kalmar/.julia/packages/PropertyT/vcGsE/src/blockdecomposition.jl:27


  1.155125 seconds (1.42 M allocations: 825.282 MiB, 6.41% gc time)


┌ Info: 
│ multiplicities  =   3  13  19  12  10   0   0   0   9  11  13  15   0   0   0   1   1   1   2   1
│     dimensions  =   1   3   3   2   1   4   8   4   6   6   6   6   4   8   4   1   3   3   2   1
└ @ PropertyT /home/kalmar/.julia/packages/PropertyT/vcGsE/src/blockdecomposition.jl:37


## Elements Adj and Op
Now we  define the elements $\operatorname{Adj}_N$ and $\operatorname{Op}_N$. The functions `Sq`, `Adj`, `Op` returning the appropriate elements are defined in the `src/sqadjop.jl` source file.

In [8]:
@time AdjN = PropertyT.Adj(RG, N)
@time OpN = PropertyT.Op(RG, N);

  0.733075 seconds (1.04 M allocations: 53.078 MiB)
  0.145846 seconds (155.32 k allocations: 7.875 MiB)


Finally we compute the element `elt` of our interest:
* if `N=3`: $\operatorname{elt} = \operatorname{Adj}_3$
* if `N=4`: $\operatorname{elt} = \operatorname{Adj}_4 + \operatorname{Op}_4$
* if `N=5`: $\operatorname{elt} = \operatorname{Adj}_5 + 1.5\operatorname{Op}_5.$

In [9]:
if N == 3
    k = 0
elseif N == 4
    k = 1
elseif N == 5
    k = 1.5
end
elt = AdjN + k*OpN;
elt.coeffs

75197-element SparseArrays.SparseVector{Int64,Int64} with 361 stored entries:
  [1    ]  =  480
  [2    ]  =  -40
  [3    ]  =  -40
  [4    ]  =  -40
  [5    ]  =  -40
  [6    ]  =  -40
  [7    ]  =  -40
  [8    ]  =  -40
  [9    ]  =  -40
  [10   ]  =  -40
           ⋮
  [418  ]  =  1
  [420  ]  =  1
  [422  ]  =  2
  [423  ]  =  1
  [424  ]  =  1
  [425  ]  =  1
  [426  ]  =  1
  [428  ]  =  1
  [429  ]  =  1
  [430  ]  =  1
  [431  ]  =  1

## Optimization Problem

We are ready to define the optimisation problem. Function
> `PropertyT.SOS_problem(x, Δ, orbit_data; upper_bound=UB)`  

defines the optimisation problem equivalent to the one of the form
\begin{align}
\text{ maximize : } \quad & \lambda\\
\text{under constraints : }\quad & 0 \leqslant \lambda \leqslant \operatorname{UB},\\
     & x - \lambda \Delta = \sum \xi_i^* \xi_i,\\
     & \text{the set of $\xi_i$s is invariant under $\mathbb{Z}/2\mathbb{Z} \wr \operatorname{Sym}_N$}.
\end{align}

In [10]:
# @time SDP_problem, varλ, varP = PropertyT.SOS_problem(elt, Δ, orbit_data)
if N == 3
    UB = 0.158
elseif N == 4
    UB = 0.82005
elseif N == 5
    UB = 1.5005
end
SDP_problem, varP = PropertyT.SOS_problem_primal(elt, Δ, block_decomposition; upper_bound=UB)

┌ Info: Adding 558 constraints...
└ @ PropertyT /home/kalmar/.julia/packages/PropertyT/vcGsE/src/sos_sdps.jl:124


  0.923033 seconds (1.47 M allocations: 331.103 MiB, 27.48% gc time)


(A JuMP Model
Maximization problem with:
Variables: 1388
Objective function type: JuMP.VariableRef
`JuMP.GenericAffExpr{Float64,JuMP.VariableRef}`-in-`MathOptInterface.EqualTo{Float64}`: 558 constraints
`Array{JuMP.GenericAffExpr{Float64,JuMP.VariableRef},1}`-in-`MathOptInterface.PositiveSemidefiniteConeSquare`: 14 constraints
`JuMP.VariableRef`-in-`MathOptInterface.LessThan{Float64}`: 1 constraint
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.
Names registered in the model: λ, Array{JuMP.VariableRef,2}[[noname noname noname; noname noname noname; noname noname noname], [noname noname … noname noname; noname noname … noname noname; … ; noname noname … noname noname; noname noname … noname noname], [noname noname … noname noname; noname noname … noname noname; … ; noname noname … noname noname; noname noname … noname noname], [noname noname … noname noname; noname noname … noname noname; … ; noname noname … noname noname; noname noname … n

In [11]:
using PropertyT.JuMP
using SCS
λ = Ps = warm = nothing

### Solving the problem
Depending on the actual problem one may need to tweak the parameters given to the solver:
 * `eps` sets the requested accuracy
 * `max_iters` sets the number of iterations to run before solver gives up
 * `alpha` is a parameter ($\alpha \in (0,2)$) which determines the rate of convergence at the cost of the accuracy
 * `acceleration_lookback`: if you experience numerical instability in scs log should be changed to `1` (at the cost of rate of convergence).
 
 The parameters below should be enough to obtain a decent solution for $\operatorname{SL}(4, \mathbb{Z}), \operatorname{SL}(5, \mathbb{Z})$.   
 For $\operatorname{SL}(3, \mathbb{Z})$ multiple runs of the following cell are required to obtain convergence (i.e. `status = MathOptInterface.OPTIMAL`; Note that when changing `UB` (above) to `0.15` a much faster convergence can be observed.

In [12]:
with_SCS = with_optimizer(SCS.Optimizer, 
    linear_solver=SCS.DirectSolver, 
    eps=3e-13,
    max_iters=10_000,
    alpha=1.5,
    acceleration_lookback=20,
    warm_start=true)

status, warm = PropertyT.solve(SDP_problem, with_SCS, warm);

λ = value(SDP_problem[:λ])
@show(status, λ);

----------------------------------------------------------------------------
	SCS v2.1.2 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
----------------------------------------------------------------------------
Lin-sys: sparse-direct, nnz in A = 133004
eps = 3.00e-13, alpha = 1.50, max_iters = 10000, normalize = 1, scale = 1.00
acceleration_lookback = 20, rho_x = 1.00e-03
Variables n = 1388, constraints m = 1946
Cones:	primal zero / dual free vars: 1196
	linear vars: 1
	sd vars: 749, sd blks: 14
Setup time: 6.60e-02s
SCS using variable warm-starting
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 1.65e+20  1.24e+20  1.00e+00 -3.16e+21  2.48e+19  3.17e+20  2.27e-03 
   100| 6.67e-05  5.27e-05  2.89e-04  6.71e-05  3.56e-04  4.04e-16  1.39e-01 
   200| 1.00e-04  7.86e-05

## Checking the solution
Now we reconstruct the solution to the original problem over $\mathbb{R} \operatorname{SL}(N,\mathbb{Z})$, which essentially boils down to averaging the obtained solution over the orbits of wreath product action:
  $$Q=\frac{1}{|\Sigma|}\sum_{\sigma\in\Sigma}\sum_{\pi\in \widehat{\Sigma}} \dim{\pi}\cdot\sigma\left(U_{\pi}^T \sqrt{P_{\pi}} U_{\pi}\right).$$

In [13]:
Ps = [value.(P) for P in varP]
Qs = real.(sqrt.(Ps));
Q = PropertyT.reconstruct(Qs, block_decomposition)

433×433 Array{Float64,2}:
  1.74178    -0.23254     -0.23254      …   0.0240788    -0.0272453
 -0.23254     3.17964     -0.0794995       -0.0263971     0.00970716
 -0.23254    -0.0794995    3.17964          0.00306073   -0.00552904
 -0.23254    -0.0794995   -0.0794995       -0.00414783    0.00668187
 -0.23254     0.0593166   -0.137798        -0.0263971     0.00970716
 -0.23254    -0.137798    -0.0797927    …   0.00306073   -0.00552904
 -0.23254    -0.137798    -0.0217825       -0.00414783    0.00668187
 -0.23254    -0.137798     0.0593166       -0.00418215    0.00675384
 -0.23254    -0.0797927   -0.137798        -0.00418215    0.00675384
 -0.23254    -0.0217825   -0.137798         0.000502018   0.00737746
 -0.23254    -0.137798    -0.137798     …   0.00315626   -0.00557801
 -0.23254    -0.0797927   -0.0217825        0.00315626   -0.00557801
 -0.23254    -0.0217825   -0.0797927       -0.00934383    0.00974041
  ⋮                                     ⋱                
  0.0625862   0.0176

As explained in the paper the columns of the square-root of the solution matrix provide the coefficients for $\xi_i$'s in basis `E_R` of the group ring. Below we compute the residual 
    $$ b = \left(x - \lambda\Delta\right) - \sum \xi_i^*\xi_i.$$
As we do it in floating-point arithmetic,  the result can't be taken seriously.

In [14]:
function SOS_residual(x::GroupRingElem, Q::Matrix)
    RG = parent(x)
    @time sos = PropertyT.compute_SOS(RG, Q);
    return x - sos
end

SOS_residual (generic function with 1 method)

In [15]:
residual = SOS_residual(elt - λ*Δ, Q)
@show norm(residual, 1);

  0.034382 seconds (43.79 k allocations: 5.102 MiB)
norm(residual, 1) = 2.1927155930872447e-9


### Checking in interval arithmetic

In [16]:
using PropertyT.IntervalArithmetic
IntervalArithmetic.setrounding(Interval, :tight)
IntervalArithmetic.setformat(sigfigs=12);

Here we resort to interval arithmetic to provide certified upper and lower bounds on the norm of the residual.
* We first change entries of `Q` to narrow intervals
* We project columns of `Q` so that $0$ is in the sum of coefficients of each column (i.e. $\xi_i \in I \operatorname{SL}(N,\mathbb{Z})$)
* We compute the sum of squares and the $\ell_1$-norm of the residual in the interval arithmetic.

The returned `check_columns_augmentation` is a boolean flag to detect if the projection was successful, i.e. if we can guarantee that each column of `Q_aug` can be represented by an element from the augmentation ideal. (If it were not successful, one may project `Q = PropertyT.augIdproj(Q)` in the floating point arithmetic prior to the cell below).

The resulting norm of the residual is **guaranteed** to be contained in the resulting interval. E.g. if each entry of `Q` were changed into an honest rational number and all the computations were carried out in rational arithmetic, the rational $\ell_1$-norm will be contained in the interval $\ell_1$-norm.

In [17]:
Q_aug, check_columns_augmentation = PropertyT.augIdproj(Interval, Q);
@assert check_columns_augmentation
elt_int = elt - @interval(λ)*Δ;
residual_int = SOS_residual(elt_int, Q_aug)
@show norm(residual_int, 1);

  0.847018 seconds (72.68 k allocations: 8.378 MiB)
norm(residual_int, 1) = [2.75348625948e-09, 3.06642022624e-09]


In [18]:
certified_λ = @interval(λ) - 2*norm(residual_int,1)

[0.820049993867, 0.820049994493]

So $\operatorname{elt} - \lambda_0 \Delta \in \Sigma^2 I\operatorname{SL}(N, \mathbb{Z})$, where as $\lambda_0$ we could take the left end of the above interval:

In [19]:
certified_λ.lo

0.820049993867084

In [20]:
using Dates
now()

2020-10-21T13:03:37.179